# Enrichment Analysis: Biological Interpretation

## Overview

This notebook demonstrates how to perform functional enrichment analysis on DNA methylation biomarkers to understand their biological significance. We use a robust fallback methodology that automatically switches between annotation databases based on availability.

### Key Components

1. **CpG to Gene Mapping**: Link methylation sites to associated genes
2. **GO Enrichment**: Gene Ontology analysis (BP, MF, CC)
3. **KEGG Pathway Analysis**: Identify enriched biological pathways
4. **Fallback Strategy**: MSigDB-based enrichment when standard tools are unavailable

### Learning Objectives

By the end of this notebook, you will be able to:

1. Map CpG probe IDs to gene symbols using EPIC array annotations
2. Perform Gene Ontology enrichment analysis
3. Identify enriched KEGG pathways
4. Use the fallback enrichment strategy for robust analysis
5. Visualize and interpret enrichment results

## 1. Environment Setup

In [ ]:
# Standard library imports
import sys
import logging
from pathlib import Path

# Scientific computing
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Project-specific imports - Enrichment Analysis
from src.enrichment import (
    EnrichmentAnalyzer,
    EnrichmentConfig,
    EnrichmentResult,
    FallbackEnrichmentStrategy,
    run_enrichment_analysis,
    GOAnnotator,
    KEGGPathwayMapper,
    MSigDBLoader,
    EPICAnnotationMapper
)

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('colorblind')

print(f"Project root: {project_root}")

## 2. Load Selected Features

In [ ]:
# Define paths
features_dir = project_root / 'data' / 'processed' / 'features'
results_dir = project_root / 'results' / 'enrichment'
figures_dir = project_root / 'data' / 'figures' / 'supplementary'

# Create output directories
results_dir.mkdir(parents=True, exist_ok=True)
figures_dir.mkdir(parents=True, exist_ok=True)

# Load different feature sets
binary_features = pd.read_csv(
    features_dir / 'binary_features_L5_moderate.csv'
)['probe_id'].tolist()

multiclass_features = pd.read_csv(
    features_dir / 'multiclass_features_L5_moderate.csv'
)['probe_id'].tolist()

# Load consensus features if available
consensus_path = features_dir / 'consensus_features.csv'
if consensus_path.exists():
    consensus_features = pd.read_csv(consensus_path)['probe_id'].tolist()
else:
    consensus_features = []

print("Loaded feature sets:")
print(f"  Binary features: {len(binary_features)}")
print(f"  Multiclass features: {len(multiclass_features)}")
print(f"  Consensus features: {len(consensus_features)}")

## 3. CpG to Gene Mapping

The Illumina EPIC array provides annotations that link each CpG probe to associated genes. We use the `EPICAnnotationMapper` to perform this mapping.

In [ ]:
# Initialize EPIC annotation mapper
# This loads the Illumina EPIC array annotation manifest
annotation_mapper = EPICAnnotationMapper(
    annotation_file=project_root / 'data' / 'external' / 'EPIC_manifest.csv'
)

print("EPIC Annotation Mapper initialized")
print(f"Annotation database loaded: {len(annotation_mapper)} probes")

In [ ]:
# Map CpG probes to genes
binary_genes = annotation_mapper.map_probes_to_genes(binary_features)
multiclass_genes = annotation_mapper.map_probes_to_genes(multiclass_features)

print("\nCpG to Gene Mapping Results:")
print(f"  Binary: {len(binary_features)} CpGs -> {len(binary_genes)} unique genes")
print(f"  Multiclass: {len(multiclass_features)} CpGs -> {len(multiclass_genes)} unique genes")

In [ ]:
# Get detailed mapping information
mapping_df = annotation_mapper.get_detailed_mapping(binary_features)

print("\nDetailed mapping example:")
print(mapping_df.head(10).to_string())

In [ ]:
# Analyze genomic feature distribution
if 'feature_type' in mapping_df.columns:
    feature_dist = mapping_df['feature_type'].value_counts()
    
    print("\nGenomic Feature Distribution:")
    for feature, count in feature_dist.items():
        print(f"  {feature}: {count} CpGs")

## 4. Initialize Enrichment Analyzer

The `EnrichmentAnalyzer` provides a unified interface for multiple enrichment methods.

In [ ]:
# Configure enrichment analysis
config = EnrichmentConfig(
    organism='Homo sapiens',
    pvalue_threshold=0.05,
    fdr_method='BH',  # Benjamini-Hochberg
    min_genes=5,      # Minimum genes per term
    max_genes=500     # Maximum genes per term
)

# Initialize analyzer
analyzer = EnrichmentAnalyzer(config=config)

print("Enrichment Analyzer Configuration:")
print(f"  Organism: {config.organism}")
print(f"  P-value threshold: {config.pvalue_threshold}")
print(f"  FDR correction: {config.fdr_method}")

## 5. Gene Ontology Enrichment Analysis

GO analysis identifies enriched biological processes (BP), molecular functions (MF), and cellular components (CC).

In [ ]:
# Initialize GO annotator
go_annotator = GOAnnotator()

print("GO Annotator initialized")
print("Available GO categories: BP (Biological Process), MF (Molecular Function), CC (Cellular Component)")

In [ ]:
# Run GO enrichment for binary features
print("\nRunning GO enrichment analysis...")

go_results = analyzer.run_go_enrichment(
    gene_list=binary_genes,
    categories=['BP', 'MF', 'CC']
)

print(f"\nGO Enrichment Results:")
for category in ['BP', 'MF', 'CC']:
    if category in go_results:
        n_terms = len(go_results[category])
        n_sig = sum(go_results[category]['padj'] < 0.05) if n_terms > 0 else 0
        print(f"  {category}: {n_sig} significant terms (FDR < 0.05)")

In [ ]:
# Display top GO terms for Biological Process
if 'BP' in go_results and len(go_results['BP']) > 0:
    bp_results = go_results['BP'].sort_values('padj')
    
    print("\nTop 10 Enriched Biological Processes:")
    print("=" * 80)
    
    display_cols = ['term_name', 'gene_count', 'padj']
    print(bp_results[display_cols].head(10).to_string(index=False))

In [ ]:
# Visualize GO enrichment results
def plot_go_barplot(results_df, category, top_n=15):
    """Create a horizontal bar plot for GO enrichment results."""
    if len(results_df) == 0:
        print(f"No results to plot for {category}")
        return None
    
    # Get top N terms
    top_terms = results_df.nsmallest(top_n, 'padj')
    
    # Create plot
    fig, ax = plt.subplots(figsize=(10, max(6, top_n * 0.4)))
    
    # Calculate -log10(p-value)
    neg_log_p = -np.log10(top_terms['padj'].values)
    
    # Create bar plot
    y_pos = np.arange(len(top_terms))
    bars = ax.barh(y_pos, neg_log_p, color='steelblue', edgecolor='black')
    
    # Set labels
    ax.set_yticks(y_pos)
    ax.set_yticklabels(top_terms['term_name'].values)
    ax.set_xlabel('-log10(Adjusted P-value)')
    ax.set_title(f'Top {len(top_terms)} GO {category} Terms')
    
    # Add significance threshold line
    ax.axvline(x=-np.log10(0.05), color='red', linestyle='--', 
               label='FDR = 0.05')
    ax.legend()
    
    plt.tight_layout()
    return fig

# Plot BP results
if 'BP' in go_results and len(go_results['BP']) > 0:
    fig = plot_go_barplot(go_results['BP'], 'Biological Process')
    if fig:
        plt.savefig(figures_dir / 'go_bp_enrichment.png', dpi=150, bbox_inches='tight')
        plt.show()

## 6. KEGG Pathway Analysis

KEGG pathway analysis identifies biological pathways that are enriched in our gene set.

In [ ]:
# Initialize KEGG pathway mapper
kegg_mapper = KEGGPathwayMapper(organism='hsa')  # Human

print("KEGG Pathway Mapper initialized")

In [ ]:
# Run KEGG pathway enrichment
print("Running KEGG pathway analysis...")

kegg_results = analyzer.run_kegg_enrichment(gene_list=binary_genes)

if len(kegg_results) > 0:
    n_sig = sum(kegg_results['padj'] < 0.05)
    print(f"\nKEGG Pathway Results:")
    print(f"  Total pathways tested: {len(kegg_results)}")
    print(f"  Significant pathways (FDR < 0.05): {n_sig}")
else:
    print("No KEGG pathway results available.")

In [ ]:
# Display top KEGG pathways
if len(kegg_results) > 0:
    kegg_sorted = kegg_results.sort_values('padj')
    
    print("\nTop 10 Enriched KEGG Pathways:")
    print("=" * 80)
    
    display_cols = ['pathway_name', 'gene_count', 'padj']
    print(kegg_sorted[display_cols].head(10).to_string(index=False))

## 7. Fallback Enrichment Strategy

When standard annotation databases are unavailable, the `FallbackEnrichmentStrategy` uses MSigDB gene sets with hypergeometric testing.

In [ ]:
# Initialize fallback strategy
fallback = FallbackEnrichmentStrategy(
    msigdb_path=project_root / 'data' / 'external' / 'msigdb'
)

print("Fallback Enrichment Strategy initialized")
print("\nAvailable gene set collections:")
for collection in fallback.available_collections:
    print(f"  - {collection}")

In [ ]:
# Run fallback enrichment analysis
print("\nRunning fallback enrichment analysis...")

fallback_results = fallback.run_analysis(
    gene_list=binary_genes,
    collections=['C5:GO:BP', 'C2:CP:KEGG', 'H']  # GO BP, KEGG, Hallmark
)

print("\nFallback Analysis Results:")
for collection, results in fallback_results.items():
    if len(results) > 0:
        n_sig = sum(results['padj'] < 0.05)
        print(f"  {collection}: {n_sig} significant gene sets")

In [ ]:
# Display Hallmark gene set results
if 'H' in fallback_results and len(fallback_results['H']) > 0:
    hallmark_results = fallback_results['H'].sort_values('padj')
    
    print("\nTop Enriched Hallmark Gene Sets:")
    print("=" * 80)
    print(hallmark_results.head(10).to_string())

## 8. Comprehensive Enrichment Analysis

Run the complete enrichment pipeline on all feature sets.

In [ ]:
# Run comprehensive analysis on binary features
print("Running comprehensive enrichment analysis on binary features...")

comprehensive_results = analyzer.run_comprehensive_analysis(
    cpg_sites=binary_features
)

print("\nComprehensive Analysis Complete:")
for analysis_type, results in comprehensive_results.items():
    if isinstance(results, dict):
        for subtype, subresults in results.items():
            if len(subresults) > 0:
                print(f"  {analysis_type}/{subtype}: {len(subresults)} terms")
    elif len(results) > 0:
        print(f"  {analysis_type}: {len(results)} terms")

## 9. Compare Feature Set Enrichments

Compare enrichment patterns between binary and multiclass feature sets.

In [ ]:
# Run enrichment on multiclass features
print("Running enrichment analysis on multiclass features...")

multi_results = analyzer.run_comprehensive_analysis(
    cpg_sites=multiclass_features
)

print("Multiclass enrichment complete.")

In [ ]:
# Compare enriched terms between feature sets
def compare_enrichment_results(results1, results2, category, name1, name2):
    """Compare enriched terms between two result sets."""
    if category not in results1 or category not in results2:
        return None
    
    # Get significant terms from each set
    sig1 = set(results1[category][results1[category]['padj'] < 0.05]['term_id'])
    sig2 = set(results2[category][results2[category]['padj'] < 0.05]['term_id'])
    
    # Calculate overlap
    overlap = sig1 & sig2
    only1 = sig1 - sig2
    only2 = sig2 - sig1
    
    print(f"\n{category} Comparison:")
    print(f"  {name1} specific: {len(only1)} terms")
    print(f"  {name2} specific: {len(only2)} terms")
    print(f"  Shared: {len(overlap)} terms")
    
    return {'overlap': overlap, 'only1': only1, 'only2': only2}

# Compare GO BP results
if 'GO' in comprehensive_results and 'GO' in multi_results:
    comparison = compare_enrichment_results(
        comprehensive_results['GO'], 
        multi_results['GO'],
        'BP',
        'Binary', 
        'Multiclass'
    )

## 10. Visualization of Enrichment Results

In [ ]:
def create_dot_plot(results_df, title, top_n=20, figsize=(10, 10)):
    """Create a dot plot for enrichment results."""
    if len(results_df) == 0:
        print("No results to plot")
        return None
    
    # Get top terms
    top_terms = results_df.nsmallest(top_n, 'padj').copy()
    
    fig, ax = plt.subplots(figsize=figsize)
    
    # Create dot plot
    y_pos = np.arange(len(top_terms))
    
    # Size by gene count, color by -log10(p-value)
    sizes = top_terms['gene_count'].values * 10
    colors = -np.log10(top_terms['padj'].values)
    
    scatter = ax.scatter(
        top_terms['gene_ratio'].values if 'gene_ratio' in top_terms.columns else colors,
        y_pos,
        s=sizes,
        c=colors,
        cmap='RdYlBu_r',
        edgecolors='black',
        linewidths=0.5
    )
    
    # Add colorbar
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('-log10(Adjusted P-value)')
    
    # Set labels
    ax.set_yticks(y_pos)
    ax.set_yticklabels(top_terms['term_name'].values)
    ax.set_xlabel('Gene Ratio' if 'gene_ratio' in top_terms.columns else '-log10(P-value)')
    ax.set_title(title)
    
    # Add size legend
    handles, labels = [], []
    for size_val in [5, 10, 20]:
        handles.append(plt.scatter([], [], s=size_val*10, c='gray', edgecolors='black'))
        labels.append(str(size_val))
    ax.legend(handles, labels, title='Gene Count', loc='lower right')
    
    plt.tight_layout()
    return fig

# Create dot plot for GO BP
if 'GO' in comprehensive_results and 'BP' in comprehensive_results['GO']:
    fig = create_dot_plot(
        comprehensive_results['GO']['BP'],
        'GO Biological Process Enrichment',
        top_n=15
    )
    if fig:
        plt.savefig(figures_dir / 'go_bp_dotplot.png', dpi=150, bbox_inches='tight')
        plt.show()

## 11. Save Enrichment Results

In [ ]:
import json

# Save gene lists
pd.DataFrame({'gene': list(binary_genes)}).to_csv(
    results_dir / 'binary_genes.csv', index=False
)
pd.DataFrame({'gene': list(multiclass_genes)}).to_csv(
    results_dir / 'multiclass_genes.csv', index=False
)

print("Gene lists saved.")

In [ ]:
# Save GO enrichment results
if 'GO' in comprehensive_results:
    for category, results in comprehensive_results['GO'].items():
        if len(results) > 0:
            output_path = results_dir / f'go_{category.lower()}_enrichment.csv'
            results.to_csv(output_path, index=False)
            print(f"Saved: {output_path.name}")

# Save KEGG results
if 'KEGG' in comprehensive_results and len(comprehensive_results['KEGG']) > 0:
    output_path = results_dir / 'kegg_pathway_enrichment.csv'
    comprehensive_results['KEGG'].to_csv(output_path, index=False)
    print(f"Saved: {output_path.name}")

In [ ]:
# Save enrichment summary
summary = {
    'binary_features': {
        'n_cpgs': len(binary_features),
        'n_genes': len(binary_genes)
    },
    'multiclass_features': {
        'n_cpgs': len(multiclass_features),
        'n_genes': len(multiclass_genes)
    },
    'enrichment_summary': {}
}

if 'GO' in comprehensive_results:
    for category, results in comprehensive_results['GO'].items():
        if len(results) > 0:
            n_sig = sum(results['padj'] < 0.05)
            summary['enrichment_summary'][f'GO_{category}'] = {
                'total_terms': len(results),
                'significant_terms': int(n_sig)
            }

if 'KEGG' in comprehensive_results and len(comprehensive_results['KEGG']) > 0:
    n_sig = sum(comprehensive_results['KEGG']['padj'] < 0.05)
    summary['enrichment_summary']['KEGG'] = {
        'total_pathways': len(comprehensive_results['KEGG']),
        'significant_pathways': int(n_sig)
    }

summary_path = results_dir / 'enrichment_summary.json'
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"\nEnrichment summary saved: {summary_path}")

## Summary

In this notebook, we performed comprehensive functional enrichment analysis:

### Key Accomplishments

1. **CpG to Gene Mapping**: Linked methylation probes to associated genes using EPIC annotations

2. **GO Enrichment Analysis**: Identified enriched biological processes, molecular functions, and cellular components

3. **KEGG Pathway Analysis**: Found enriched biological pathways

4. **Fallback Strategy**: Demonstrated robust analysis using MSigDB gene sets when standard tools are unavailable

5. **Feature Set Comparison**: Compared enrichment patterns between binary and multiclass features

### Biological Insights

The enrichment analysis reveals the biological processes and pathways underlying HIIT-induced methylation changes. Key themes often include:

- Metabolic regulation
- Muscle adaptation and development
- Energy metabolism pathways
- Inflammatory response modulation

### Next Steps

Review the **examples/** notebooks for:
- Quick start guide to the complete pipeline
- Customization options for your own analyses

In [ ]:
# Session summary
print("=" * 60)
print("ENRICHMENT ANALYSIS COMPLETE")
print("=" * 60)
print(f"\nBinary features:")
print(f"  CpGs: {len(binary_features)} -> Genes: {len(binary_genes)}")
print(f"\nMulticlass features:")
print(f"  CpGs: {len(multiclass_features)} -> Genes: {len(multiclass_genes)}")
print(f"\nResults saved to: {results_dir}")